In [ ]:
!pip install fosforio[snowflake]

In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from fosforml import *
from fosforml.constants import MLModelFlavours

/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
from refractio import snowflake
cars = snowflake.get_dataframe("CAR_NEW100MB")
cars

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Connection details fetched: {'errorCode': 'CNM-404', 'message': 'Asset not found', 'detailMessage': 'Datasource not found by name CAR_NEW100MB', 'timeStamp': '2024-06-27 07:40:51.76', 'api': '/v1/external/getConnConfig/{dataSourceName}/{userId}/{projectId}'}
Ex: 'params'
Exception occurred in reading data_frame from snowflake connection: Exception occurred in creating snowflake connection: Datasource not found by name CAR_NEW100MB


In [ ]:
# cars = pd.read_csv("/data/car_new.csv")
# cars

In [ ]:
Y = cars['CLASS_NUM']
X = cars.drop (['CLASS_NUM'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

In [ ]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [ ]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [ ]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

In [ ]:
register_model(model, 
               score, 
               name='car_evaluation_new2', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="",
               model_type="classification",
               input_type="json", 
               explain_ai=True, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=False, 
               kyd_score = False
              )